Vamos a armar la red completa con:
    
    * Genes y sus interacciones de StringDB (score > 0.8)
    * Variantes y su mapeo en genes
    * Fenotipos y sus asociaciones con variantes
    * Categorías generales de fenotipos

In [48]:
%run imports.py

columnas = [
    "gen_mapeado_elegido",
    "alelo_riesgo",
    # "OR_or_beta",
    "fenotipo",
    "categoria_fenotipo"
]

assoc = pd.read_table("results/gwas_cat.filtrado.tsv.gz", usecols=columnas)[columnas]
assoc = assoc.sort_values(by=["categoria_fenotipo", "fenotipo", "gen_mapeado_elegido"])
assoc = assoc.reset_index(drop=True)

print(f"{len(assoc):,} asociaciones")

assoc.sample(10)

112,626 asociaciones


,gen_mapeado_elegido,alelo_riesgo,fenotipo,categoria_fenotipo
72352,DEPTOR,rs79346194-A,Estimated glomerular filtration rate,Other measurement
74820,NaN,rs1366594-A,Femoral neck bone mineral density,Other measurement
6584,SDK1,rs959894-A,Self-reported math ability (MTAG),Biological process
89096,VARS,rs3115670-T,Mosquito bite size,Other measurement
59002,MGAT5,rs62165726-A,Blood protein levels,Other measurement
7415,RUNX1T1,rs6986430-C,Smoking initiation (ever regular vs never regular),Biological process
28043,STEAP1B,rs76659285-?,Mean corpuscular hemoglobin,Hematological measurement
83374,EEFSEC,rs2811415-G,Lung function (FEV1/FVC),Other measurement
6948,NaN,rs7156431-C,Self-reported math ability (MTAG),Biological process
81586,ARNTL,rs6486121-T,Iron status biomarkers (transferrin levels),Other measurement


In [49]:
prot_links = pd.read_table("results/prot_links.stringdb.tsv.gz")
print(f"{len(prot_links):,} links entre proteínas")

prot_links.sample(5)

383,408 links entre proteínas


,protein_1,protein_2
374976,CLTC,PACSIN2
378210,DDX24,NSA2
162446,SRSF7,NUDT21
216853,PSMB5,PSMD14
265206,SIN3B,SAP30BP


In [50]:
# * Genes y sus interacciones de StringDB (score > 0.8)
enlaces_gen_gen = zip(prot_links.protein_1, prot_links.protein_2)

# * Variantes y su mapeo en genes
enlaces_variante_gen = (
    assoc[["gen_mapeado_elegido", "alelo_riesgo"]]   
    .dropna()
    .drop_duplicates()
    .set_index("alelo_riesgo")
    .gen_mapeado_elegido
    .items()
)

# * Fenotipos y sus asociaciones con variantes
enlaces_variante_fenotipo = (
    assoc[["alelo_riesgo", "fenotipo"]]
    .dropna()
    .drop_duplicates()
    .set_index("alelo_riesgo")
    .fenotipo
    .items()
)
    
# * Categorías generales de fenotipos
enlaces_fenotipo_categoria = (
    assoc[["fenotipo", "categoria_fenotipo"]]
    .dropna()
    .drop_duplicates()
    .set_index("fenotipo")
    .categoria_fenotipo
    .items()
)

red = nx.Graph()

red.add_edges_from(enlaces_gen_gen, tipo="PPI")
red.add_edges_from(enlaces_variante_gen, tipo="mapeo")
red.add_edges_from(enlaces_variante_fenotipo, tipo="asociacion")
red.add_edges_from(enlaces_fenotipo_categoria, tipo="ontologia")

print(f"{len(red):,} nodos en la red")

113,010 nodos en la red


In [70]:
genes_observados = set(prot_links.protein_1) | set(prot_links.protein_2) | set(assoc.gen_mapeado_elegido.dropna())
print(f"{len(genes_observados):,} genes observados")

tipos_de_nodo = dict(
    [(item, "gen") for item in genes_observados] +
    [(item, "alelo") for item in assoc.alelo_riesgo] +
    [(item, "fenotipo") for item in assoc.fenotipo] +
    [(item, "categoria_fenotipo") for item in assoc.categoria_fenotipo]
)

nx.set_node_attributes(red, tipos_de_nodo, name="tipo")

15,988 genes observados


In [72]:
fp = "results/red_completa.gml" 
nx.write_gml(red, fp)

print(fp)
!ls -lh $fp

results/red_completa.gml
-rw-r--r-- 1 juan juan 29M Jul  7 19:50 results/red_completa.gml
